# DRNN LEARNING

##  Desafios no Treinamento da FDRNN

###  Problema apontado

Embora a formulação apresentada na equação (8) seja **conceitualmente elegante**, ela traz uma **grande dificuldade prática**:

> A otimização do modelo se torna **muito complexa**, pois envolve uma rede neural profunda com **milhares de parâmetros ocultos (latentes)**.

Em outras palavras:
- O modelo FDRNN combina três componentes:
  - Representação fuzzy;
  - Transformação profunda (Deep Neural Network);
  - Política de decisão recorrente.
- Isso resulta em uma arquitetura grande e interdependente.
- Otimizar tudo isso de uma vez, diretamente, é computacionalmente difícil e propenso a instabilidade.

---

### Solução proposta

Para contornar essa dificuldade, os autores propõem uma **estratégia de treinamento prática** baseada em **duas etapas principais**:

1. **Inicialização do sistema (System Initialization)**
   - Treinar separadamente partes da rede para obter parâmetros razoáveis antes do treinamento conjunto.

2. **Ajuste fino (Fine Tuning)**
   - Refinar todos os parâmetros juntos de forma mais precisa e específica para a tarefa final, após uma boa inicialização.

Essa abordagem divide o problema em etapas mais gerenciáveis, facilitando o treinamento da rede como um todo.

---

###  Intenção dessa seção

A partir daqui, a **Seção IV** vai detalhar **como implementar esse treinamento em duas fases**, explicando como lidar com a recorrência, com os gradientes temporais, e como evitar problemas como o desaparecimento ou explosão do gradiente.



##  Inicialização do Sistema

A inicialização dos parâmetros é **uma etapa crítica** no treinamento de redes neurais profundas (DNNs), especialmente quando se trabalha com estruturas complexas como o FDRNN. Esta seção descreve como inicializar **cada uma das três partes do modelo**:  
1. Representação Fuzzy  
2. Transformação Profunda (Deep Learning)  
3. Política de Trading (DRL)

---

### 1.  Inicialização da Representação Fuzzy

- Cada entrada do vetor ${\bf f}_t$ é conectada a **três funções de pertinência fuzzy**.
- Cada função fuzzy tem dois parâmetros:
  - Centro $m_i$
  - Largura (variância) $σ_i²$

#### Estratégia:
- Utiliza-se o algoritmo **k-means** para dividir os dados de treino em **k = 3 grupos**:
  - Correspondentes às três categorias: tendência de alta, baixa e neutra.
- Em cada cluster, calcula-se a **média e a variância** de cada dimensão do vetor ${\bf f}_t$.
- Esses valores são usados como inicialização dos **centros e larguras das funções fuzzy**.

---

### 2.  Inicialização da Transformação Profunda (Rede Densa)

Para inicializar a rede profunda, os autores utilizam uma técnica clássica de **aprendizado não supervisionado** chamada **Autoencoder (AE)**.

#### O que é um Autoencoder?
- Um AE tenta **reconstruir a entrada** após passá-la por uma ou mais camadas ocultas.
- Serve para aprender **representações úteis (codificações)** sem usar rótulos.

#### Como é usado aqui:
- Define-se três camadas:
  - Camada de entrada ($l$)
  - Camada oculta ($l + 1$)
  - Camada de reconstrução ($l + 2$)
- Aplica-se uma transformação $h_θ(·)$ da camada $l$ para a $l+1$ e $h_γ(·)$ da $l+1$ para a $l+2$.

#### Função de perda do AE:
A minimização é feita sobre:



$$Loss = || {\bf x}_t − h_γ(h_θ({\bf x}_t)) ||² + η * || {\bf w}_(l+1) ||²$$


- O primeiro termo mede o erro de reconstrução (diferença entre entrada e saída).
- O segundo termo é uma **penalização de regularização (L2)** para evitar overfitting.

#### Resultado:
- Após treinar o AE, os **parâmetros da camada oculta $l+1$ (θ = {w, b})** são salvos e usados como inicialização da rede profunda.
- Os parâmetros da camada de reconstrução ($γ$) **são descartados** — ela é apenas uma estrutura auxiliar.

#### Processo:
- Esse procedimento é **repetido camada por camada**, até que todas as camadas da rede profunda estejam inicializadas.

---

### 3.  Inicialização da Parte de Decisão (DRL)

- Após obter a representação profunda $F_t$ com o DNN inicializado, usa-se essa entrada para **treinar a parte de decisão** ($δ_t$).
- Isso equivale a treinar uma **rede rasa recorrente** (como na Fig. 1(a)), já conhecida da literatura.

---

###  Conclusão da Inicialização

Todos os procedimentos descritos até aqui tratam apenas de **inicializações**.  
No entanto, para que o sistema como um todo seja eficaz, ainda é necessário um **ajuste fino (fine tuning)**.

Esse ajuste final:
- Ajusta os parâmetros de **todas as partes da rede**.
- É feito de forma **sensível à tarefa final** (no caso, maximizar retorno).
- Pode ser visto como um processo de **aprendizado de representações dependente da tarefa** (task-dependent feature learning).

A próxima seção abordará justamente esse passo: **como treinar a rede recorrente profunda com backpropagation através do tempo (BPTT)** de maneira adequada.
